# Day 5: The MDL Principle - Interactive Notebook

**Paper:** "A Tutorial Introduction to the Minimum Description Length Principle" - Grünwald (2004)

---

## What You'll Learn

1. **The Core Idea:** The best model is the one that gives the shortest total description of the data
2. **Two-Part Codes:** Model description + data-given-model description
3. **Prequential MDL:** Sequential prediction coding
4. **MDL vs AIC vs BIC:** How they compare on polynomial model selection
5. **Practical Application:** Polynomial degree selection (the paper's running example)

---

## The Coding Framework (Section 3 of the paper)

Imagine you need to transmit a sequence of temperature readings to a receiver:

- **Naive approach:** Send each number individually (expensive!)
- **Smart approach:** Send a formula + small corrections (cheap!)

The **MDL Principle** says: Find the hypothesis that minimizes total description length.

In [ ]:
# Setup - Import libraries
import numpy as np
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

# Plotting style
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

print("[OK] Libraries imported.")

## Part 1: Understanding Two-Part Codes

The fundamental MDL equation (Section 3):

$$L_{\text{total}} = L(H) + L(D|H)$$

Where:
- $L(H)$ = bits to describe the **model** (hypothesis)
- $L(D|H)$ = bits to describe the **data given the model** (residuals)

The best model minimizes this sum.

In [ ]:
# Universal code for integers (fair encoding with unknown range)
def universal_code_length(n: int) -> float:
    """Bits needed to encode positive integer n."""
    if n <= 0:
        return float('inf')
    
    c0 = 2.865064  # Normalization constant
    length = np.log2(c0)
    current = float(n)
    
    while current > 1:
        length += np.log2(current)
        current = np.log2(current)
        if current <= 0:
            break
    
    return max(length, 0)

# Test it
for n in [1, 2, 10, 100, 1000]:
    print(f"Encoding integer {n}: {universal_code_length(n):.2f} bits")

In [ ]:
def two_part_mdl(x, y, degree, precision=32):
    """
    Compute Two-Part MDL for polynomial regression.
    
    Returns: (total_mdl, model_cost, data_cost)
    """
    n = len(x)
    
    # Fit polynomial
    coeffs = np.polyfit(x, y, degree)
    poly = np.poly1d(coeffs)
    residuals = y - poly(x)
    
    # === PART 1: Model Cost L(H) ===
    # Cost to specify degree
    degree_cost = universal_code_length(degree + 1)
    # Cost to specify coefficients
    coeff_cost = (degree + 1) * (precision + 1)  # +1 for sign
    model_cost = degree_cost + coeff_cost
    
    # === PART 2: Data Cost L(D|H) ===
    # Encode residuals using Gaussian model
    sigma = np.std(residuals) if np.std(residuals) > 1e-10 else 1e-10
    
    # Negative log-likelihood in bits
    nll = 0.5 * n * np.log2(2 * np.pi * sigma**2) + \
          np.sum(residuals**2) / (2 * sigma**2 * np.log(2))
    
    # Add cost to encode sigma
    sigma_cost = precision + 10
    data_cost = nll + sigma_cost
    
    return model_cost + data_cost, model_cost, data_cost

print("[OK] MDL function defined.")

## Part 2: Generate Synthetic Data

Let's create polynomial data with known structure to test MDL.

In [ ]:
# Generate data with TRUE degree = 2 (parabola)
TRUE_DEGREE = 2
N_SAMPLES = 50
NOISE_STD = 2.0

x = np.linspace(0, 10, N_SAMPLES)

# True polynomial: y = 3 + 2x - 0.5x²
true_coeffs = [-0.5, 2, 3]  # [x², x, constant]
y_true = 3 + 2*x - 0.5*x**2
y = y_true + np.random.randn(N_SAMPLES) * NOISE_STD

# Plot the data
plt.figure(figsize=(10, 5))
plt.scatter(x, y, c='gray', alpha=0.7, label='Noisy data')
plt.plot(x, y_true, 'g-', linewidth=2, label=f'True (degree {TRUE_DEGREE})')
plt.xlabel('x')
plt.ylabel('y')
plt.title(f'Synthetic Data: True Degree = {TRUE_DEGREE}')
plt.legend()
plt.show()

print(f"Generated {N_SAMPLES} points with noise std = {NOISE_STD}")

## Part 3: MDL Model Selection

Now let's compute MDL scores for different polynomial degrees and see if it recovers the true degree.

In [ ]:
# Compute MDL for degrees 0 through 10
max_degree = 10
degrees = list(range(max_degree + 1))

mdl_scores = {}
model_costs = {}
data_costs = {}

print("MDL Scores by Degree:")
print("-" * 60)
print(f"{'Degree':<8} {'Model L(H)':<15} {'Data L(D|H)':<15} {'Total':<15}")
print("-" * 60)

for deg in degrees:
    total, model, data = two_part_mdl(x, y, deg)
    mdl_scores[deg] = total
    model_costs[deg] = model
    data_costs[deg] = data
    
    marker = " <-- BEST" if deg == min(mdl_scores, key=mdl_scores.get) else ""
    print(f"{deg:<8} {model:<15.1f} {data:<15.1f} {total:<15.1f}{marker}")

best_degree = min(mdl_scores, key=mdl_scores.get)
print("-" * 60)
print(f"\nMDL selects: Degree {best_degree}")
print(f"True degree: {TRUE_DEGREE}")
print(f"{'[OK] Correct!' if best_degree == TRUE_DEGREE else '[FAIL] Incorrect'}")

In [ ]:
# Visualize the MDL breakdown
fig, ax = plt.subplots(figsize=(12, 6))

x_pos = np.array(degrees)
model_vals = [model_costs[d] for d in degrees]
data_vals = [data_costs[d] for d in degrees]
total_vals = [mdl_scores[d] for d in degrees]

# Stacked bars
ax.bar(x_pos, model_vals, label='Model Cost L(H)', color='#E94F37', alpha=0.8)
ax.bar(x_pos, data_vals, bottom=model_vals, label='Data Cost L(D|H)', 
       color='#44AF69', alpha=0.8)

# Total line
ax.plot(x_pos, total_vals, 'o-', color='purple', linewidth=2.5, 
        markersize=8, label='Total MDL')

# Mark best and true
ax.axvline(x=best_degree, color='purple', linestyle='--', linewidth=2, 
           alpha=0.7, label=f'Best (Degree {best_degree})')
ax.axvline(x=TRUE_DEGREE, color='green', linestyle=':', linewidth=2,
           alpha=0.7, label=f'True (Degree {TRUE_DEGREE})')

ax.set_xlabel('Polynomial Degree', fontsize=12)
ax.set_ylabel('Code Length (bits)', fontsize=12)
ax.set_title('MDL Score Breakdown: Model Cost vs Data Cost', fontsize=14)
ax.legend(loc='upper right')
ax.set_xticks(degrees)

plt.tight_layout()
plt.show()

## Part 4: Visual Comparison of Fits

Let's see what different polynomial degrees look like.

In [ ]:
# Compare fits: Underfit, Optimal, Overfit
degrees_to_show = [1, 2, 5, 9]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))
x_fine = np.linspace(x.min(), x.max(), 200)

for ax, deg in zip(axes, degrees_to_show):
    # Plot data
    ax.scatter(x, y, c='gray', alpha=0.6, s=30)
    
    # Fit polynomial
    coeffs = np.polyfit(x, y, deg)
    y_fit = np.poly1d(coeffs)(x_fine)
    
    # Color based on optimality
    if deg == best_degree:
        color = 'green'
        status = '[ok] OPTIMAL'
    elif deg < best_degree:
        color = 'orange'
        status = 'Underfit'
    else:
        color = 'red'
        status = 'Overfit'
    
    ax.plot(x_fine, y_fit, color=color, linewidth=2.5)
    ax.set_title(f'Degree {deg}\n{status}\nMDL: {mdl_scores[deg]:.0f} bits')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

plt.suptitle('Polynomial Fits: From Underfit to Overfit', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Part 5: MDL vs AIC vs BIC

Comparing the three most popular model selection criteria (see Grünwald Section 8):

| Criterion | Formula | Penalty |
|-----------|---------|---------|
| MDL | $L(H) + L(D\|H)$ | Information-theoretic |
| AIC | $2k - 2\ln(L)$ | Fixed: 2 per parameter |
| BIC | $k\ln(n) - 2\ln(L)$ | Grows with data size |

In [ ]:
def aic(x, y, degree):
    """Akaike Information Criterion."""
    n = len(x)
    k = degree + 2  # coefficients + variance
    
    coeffs = np.polyfit(x, y, degree)
    residuals = y - np.poly1d(coeffs)(x)
    sigma_ml = np.sqrt(np.sum(residuals**2) / n)
    
    log_lik = -0.5 * n * (np.log(2 * np.pi * sigma_ml**2) + 1)
    return 2 * k - 2 * log_lik

def bic(x, y, degree):
    """Bayesian Information Criterion."""
    n = len(x)
    k = degree + 2
    
    coeffs = np.polyfit(x, y, degree)
    residuals = y - np.poly1d(coeffs)(x)
    sigma_ml = np.sqrt(np.sum(residuals**2) / n)
    
    log_lik = -0.5 * n * (np.log(2 * np.pi * sigma_ml**2) + 1)
    return k * np.log(n) - 2 * log_lik

# Compute all scores
aic_scores = {d: aic(x, y, d) for d in degrees}
bic_scores = {d: bic(x, y, d) for d in degrees}

# Find best for each
mdl_best = min(mdl_scores, key=mdl_scores.get)
aic_best = min(aic_scores, key=aic_scores.get)
bic_best = min(bic_scores, key=bic_scores.get)

print("MODEL SELECTION COMPARISON")
print("=" * 40)
print(f"True degree:  {TRUE_DEGREE}")
print(f"MDL selects:  {mdl_best} {'[ok]' if mdl_best == TRUE_DEGREE else '[FAIL]'}")
print(f"AIC selects:  {aic_best} {'[ok]' if aic_best == TRUE_DEGREE else '[FAIL]'}")
print(f"BIC selects:  {bic_best} {'[ok]' if bic_best == TRUE_DEGREE else '[FAIL]'}")

In [ ]:
# Visualize comparison
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

methods = [
    ('MDL', mdl_scores, mdl_best, '#2E86AB'),
    ('AIC', aic_scores, aic_best, '#A23B72'),
    ('BIC', bic_scores, bic_best, '#F18F01')
]

for ax, (name, scores, best, color) in zip(axes, methods):
    vals = [scores[d] for d in degrees]
    
    ax.bar(degrees, vals, color=color, alpha=0.7)
    ax.axvline(x=best, color='black', linestyle='--', linewidth=2,
               label=f'Selected: {best}')
    ax.axvline(x=TRUE_DEGREE, color='green', linestyle=':', linewidth=2,
               label=f'True: {TRUE_DEGREE}')
    
    ax.set_xlabel('Polynomial Degree')
    ax.set_ylabel('Score')
    ax.set_title(f'{name}: Selects Degree {best}')
    ax.legend()

plt.suptitle('MDL vs AIC vs BIC: Who Wins?', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## Part 6: Compression and Model Quality

The key insight from Section 3: **A model that achieves shorter total description length has captured more of the data's regularity.**

In [ ]:
# Compression analysis
raw_bits = N_SAMPLES * 32  # 32-bit floats
best_mdl = mdl_scores[best_degree]
compression_ratio = raw_bits / best_mdl

print("COMPRESSION ANALYSIS")
print("=" * 40)
print(f"Raw data:        {raw_bits} bits ({N_SAMPLES} x 32-bit)")
print(f"MDL compressed:  {best_mdl:.0f} bits")
print(f"Compression:     {compression_ratio:.2f}x")
print()
if compression_ratio > 1:
    print("[OK] Data has compressible structure.")
    print("   The model found a pattern that reduces description length.")
else:
    print("[NOTE] Data might be random (no compressible structure).")

# Visualize
fig, ax = plt.subplots(figsize=(8, 5))

compression_ratios = [raw_bits / mdl_scores[d] for d in degrees]
colors = ['green' if d == best_degree else '#2E86AB' for d in degrees]

ax.bar(degrees, compression_ratios, color=colors, alpha=0.8)
ax.axhline(y=1.0, color='red', linestyle='--', linewidth=2, label='No compression')
ax.set_xlabel('Polynomial Degree')
ax.set_ylabel('Compression Ratio')
ax.set_title('Compression Ratio by Model Complexity\n(Higher = Better)')
ax.legend()

plt.tight_layout()
plt.show()

## Part 7: Monte Carlo - Testing Robustness

Let's run many experiments to see how reliably MDL finds the true degree.

In [ ]:
# Run 100 experiments
N_TRIALS = 100

mdl_correct = 0
aic_correct = 0
bic_correct = 0

for trial in range(N_TRIALS):
    # Generate new data
    np.random.seed(trial)
    y_trial = y_true + np.random.randn(N_SAMPLES) * NOISE_STD
    
    # Compute scores
    trial_mdl = {d: two_part_mdl(x, y_trial, d)[0] for d in degrees}
    trial_aic = {d: aic(x, y_trial, d) for d in degrees}
    trial_bic = {d: bic(x, y_trial, d) for d in degrees}
    
    # Check if correct
    mdl_correct += (min(trial_mdl, key=trial_mdl.get) == TRUE_DEGREE)
    aic_correct += (min(trial_aic, key=trial_aic.get) == TRUE_DEGREE)
    bic_correct += (min(trial_bic, key=trial_bic.get) == TRUE_DEGREE)

print(f"MONTE CARLO RESULTS ({N_TRIALS} trials)")
print("=" * 40)
print(f"MDL accuracy: {mdl_correct}/{N_TRIALS} ({100*mdl_correct/N_TRIALS:.1f}%)")
print(f"AIC accuracy: {aic_correct}/{N_TRIALS} ({100*aic_correct/N_TRIALS:.1f}%)")
print(f"BIC accuracy: {bic_correct}/{N_TRIALS} ({100*bic_correct/N_TRIALS:.1f}%)")

# Bar chart
fig, ax = plt.subplots(figsize=(8, 5))
methods = ['MDL', 'AIC', 'BIC']
accuracies = [mdl_correct, aic_correct, bic_correct]
colors = ['#2E86AB', '#A23B72', '#F18F01']

bars = ax.bar(methods, accuracies, color=colors, alpha=0.8)
ax.set_ylabel('Correct Selections')
ax.set_title(f'Model Selection Accuracy ({N_TRIALS} trials)')
ax.set_ylim(0, N_TRIALS)

for bar, acc in zip(bars, accuracies):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
            f'{100*acc/N_TRIALS:.1f}%', ha='center', fontsize=12)

plt.tight_layout()
plt.show()

## Key Takeaways

### 1. The MDL Principle
> The best hypothesis is the one that yields the shortest total description of the data (Grünwald, Section 1).

### 2. Two-Part Code
> Total cost = Model description + Data-given-model description  
> $L(H) + L(D|H)$

### 3. Trade-off
- **Simple model**: Cheap to describe, but poor fit (high residual cost)
- **Complex model**: Expensive to describe, but good fit (low residual cost)
- **Optimal model**: Minimizes the **sum**

### 4. Compression and Regularity
If you can describe data more briefly than the raw encoding, your model has captured regularity in the data. Random noise cannot be compressed (Section 3.4).

### 5. MDL vs Others (Section 8)
- **AIC**: Fixed penalty (2 per parameter) - can select overly complex models with large samples
- **BIC**: log(n) penalty - equivalent to two-part MDL under certain conditions
- **MDL**: Information-theoretic foundation - no arbitrary penalty constants

## Next Steps

1. **Try the exercises** in the `Exercises/` folder
2. **Run `train_minimal.py`** with different parameters
3. **Explore `visualization.py`** for more plots
4. **Read the paper** for mathematical details on prequential MDL and NML